In [18]:
import cv2
import numpy as np
import os
from glob import glob

# ==============================
# CONFIG
# ==============================
INPUT_DIR = r"D:\projects\neuro\all fascicles\fascicle images\set2_Image_01_20x_bf_05_all"
OUTPUT_DIR = r"C:\Users\DELL\Downloads\blur_maps_set2_Image_01_20x_bf_05_all_0"
WINDOW_SIZES = [3, 5, 7]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================
# Function: local Laplacian variance
# ==============================
def local_laplacian_variance(lap_img, k):
    mean = cv2.blur(lap_img, (k, k))
    mean_sq = cv2.blur(lap_img ** 2, (k, k))
    var = mean_sq - mean ** 2
    var = cv2.normalize(var, None, 0, 255, cv2.NORM_MINMAX)
    return var.astype(np.uint8)

# ==============================
# Process all PNG images
# ==============================
png_files = glob(os.path.join(INPUT_DIR, "*.png"))

if len(png_files) == 0:
    raise RuntimeError("No PNG images found in input directory!")

for img_path in png_files:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    print(f"Processing: {filename}.png")

    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read {img_path}, skipping.")
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Output subfolder per image
    img_out_dir = os.path.join(OUTPUT_DIR, filename)
    os.makedirs(img_out_dir, exist_ok=True)

    # --------------------------
    # Save Gaussian low-pass outputs
    # --------------------------
    # for k in WINDOW_SIZES:
    #     gaussian = cv2.GaussianBlur(gray, (k, k), 0)
    #     gauss_path = os.path.join(img_out_dir, f"gaussian_{k}x{k}.png")
    #     cv2.imwrite(gauss_path, gaussian)

    # --------------------------
    # Denoising for Laplacian
    # --------------------------
    gray_denoised = cv2.GaussianBlur(gray, (3, 3), 0)

    # Laplacian
    lap = cv2.Laplacian(gray_denoised, cv2.CV_64F)
    lap = np.abs(lap)

    # --------------------------
    # Blur maps (multi-scale)
    # --------------------------
    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        out_path = os.path.join(img_out_dir, f"blur_map_{k}x{k}.png")
        cv2.imwrite(out_path, blur_map)

print("✅ Done processing all PNG images.")
#blur maps only

Processing: 1.png
Processing: 2.png
Processing: 3.png
Processing: 4.png
Processing: 5.png
✅ Done processing all PNG images.


In [5]:
import cv2
import numpy as np
import os
from glob import glob

# ==============================
# CONFIG
# ==============================
INPUT_DIR = r"D:\projects\neuro\all fascicles\fascicle images\set2_Image_01_20x_bf_05_all"
OUTPUT_DIR = r"C:\Users\DELL\Downloads\blur_maps_set2_Image_01_20x_bf_05_all"
WINDOW_SIZES = [3, 5, 7]

PERCENTILE_THRESHOLD = 30   # lower = stricter blur
MORPH_KERNEL_SIZE = 5

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================
# Local Laplacian variance
# ==============================
def local_laplacian_variance(lap_img, k):
    mean = cv2.blur(lap_img, (k, k))
    mean_sq = cv2.blur(lap_img ** 2, (k, k))
    var = mean_sq - mean ** 2
    var = cv2.normalize(var, None, 0, 255, cv2.NORM_MINMAX)
    return var.astype(np.uint8)

# ==============================
# Blur mask from blur map
# ==============================
def get_blur_mask(blur_map, percentile=30):
    thresh = np.percentile(blur_map, percentile)
    mask = np.zeros_like(blur_map, dtype=np.uint8)
    mask[blur_map <= thresh] = 255   # blurred = keep
    return mask

# ==============================
# Process images
# ==============================
png_files = glob(os.path.join(INPUT_DIR, "*.png"))

if len(png_files) == 0:
    raise RuntimeError("❌ No PNG images found!")

for img_path in png_files:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    print(f"Processing: {filename}.png")

    # --------------------------
    # Read image
    # --------------------------
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read {img_path}, skipping.")
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Output folder per image
    img_out_dir = os.path.join(OUTPUT_DIR, filename)
    os.makedirs(img_out_dir, exist_ok=True)

    # --------------------------
    # Denoise before Laplacian
    # --------------------------
    gray_denoised = cv2.GaussianBlur(gray, (3, 3), 0)

    # --------------------------
    # Laplacian (high-pass)
    # --------------------------
    lap = cv2.Laplacian(gray_denoised, cv2.CV_64F)
    lap = np.abs(lap)

    # --------------------------
    # Multi-scale blur maps
    # --------------------------
    for k in WINDOW_SIZES:
        # Blur map
        blur_map = local_laplacian_variance(lap, k)
        cv2.imwrite(os.path.join(img_out_dir, f"blur_map_{k}x{k}.png"), blur_map)

        # --------------------------
        # Blur-only mask
        # --------------------------
        blur_mask = get_blur_mask(blur_map, PERCENTILE_THRESHOLD)

        # Morphological cleanup
        kernel = np.ones((MORPH_KERNEL_SIZE, MORPH_KERNEL_SIZE), np.uint8)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_OPEN, kernel)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_CLOSE, kernel)

        cv2.imwrite(os.path.join(img_out_dir, f"blur_mask_{k}x{k}.png"), blur_mask)

        # --------------------------
        # Apply mask to original image
        # --------------------------
        blur_only_fascicle = cv2.bitwise_and(img, img, mask=blur_mask)
        cv2.imwrite(
            os.path.join(img_out_dir, f"blur_only_fascicle_{k}x{k}.png"),
            blur_only_fascicle
        )

print("✅ Done processing all PNG images.")
#has mask too but spotty

Processing: 1.png
Processing: 2.png
Processing: 3.png
Processing: 4.png
Processing: 5.png
✅ Done processing all PNG images.


In [12]:
def tissue_mask(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv)

    sat_mask = cv2.inRange(S, 15, 255)
    not_too_bright = cv2.inRange(V, 0, 250)

    combined = cv2.bitwise_and(sat_mask, not_too_bright)

    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    opened = cv2.morphologyEx(combined, cv2.MORPH_OPEN, k, iterations=1)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, k, iterations=2)

    return closed


In [13]:
def local_laplacian_variance(lap_img, k):
    mean = cv2.blur(lap_img, (k, k))
    mean_sq = cv2.blur(lap_img ** 2, (k, k))
    var = mean_sq - mean ** 2
    var = cv2.normalize(var, None, 0, 255, cv2.NORM_MINMAX)
    return var.astype(np.uint8)


def get_blur_mask(blur_map, percentile):
    thresh = np.percentile(blur_map, percentile)
    mask = np.zeros_like(blur_map, dtype=np.uint8)
    mask[blur_map <= thresh] = 255
    return mask


In [16]:
WINDOW_SIZES = [3, 5, 7]
PERCENTILE_THRESHOLD = 5   # tighter for tissue-only
MORPH_KERNEL_SIZE = 25

for img_path in glob(os.path.join(INPUT_DIR, "*.png")):
    name = os.path.splitext(os.path.basename(img_path))[0]
    print("Processing:", name)

    img = cv2.imread(img_path)
    if img is None:
        continue

    # ==============================
    # STEP 1: TISSUE MASKING
    # ==============================
    tissue = tissue_mask(img)
    tissue_only = cv2.bitwise_and(img, img, mask=tissue)

    # Save tissue-only image
    out_dir = os.path.join(r"D:\projects\neuro\lowfreq", name)
    os.makedirs(out_dir, exist_ok=True)
    cv2.imwrite(os.path.join(out_dir, "01_tissue_only.png"), tissue_only)

    # ==============================
    # STEP 2: BLUR MAP ON TISSUE ONLY
    # ==============================
    gray = cv2.cvtColor(tissue_only, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap = np.abs(lap)

    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        cv2.imwrite(os.path.join(out_dir, f"02_blur_map_{k}x{k}.png"), blur_map)

        # ==============================
        # STEP 3: LOW-FREQ (BLUR) MASK
        # ==============================
        blur_mask = get_blur_mask(blur_map, PERCENTILE_THRESHOLD)

        kernel = np.ones((MORPH_KERNEL_SIZE, MORPH_KERNEL_SIZE), np.uint8)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_CLOSE, kernel)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_OPEN, kernel)

        cv2.imwrite(os.path.join(out_dir, f"03_blur_mask_{k}x{k}.png"), blur_mask)

        # ==============================
        # STEP 4: FINAL OVERLAY (CORRECT)
        # ==============================
        final_mask = cv2.bitwise_and(tissue, blur_mask)
        
        final_low_freq = cv2.bitwise_and(img, img, mask=final_mask)
        
        cv2.imwrite(
            os.path.join(out_dir, f"04_low_freq_only_{k}x{k}.png"),
            final_low_freq
        )

#tissue mask->blurmaps

Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5


In [17]:
WINDOW_SIZES = [3, 5, 7]
PERCENTILE_THRESHOLD = 5
MORPH_KERNEL_SIZE = 25

for img_path in glob(os.path.join(INPUT_DIR, "*.png")):
    name = os.path.splitext(os.path.basename(img_path))[0]
    print("Processing:", name)

    img = cv2.imread(img_path)
    if img is None:
        continue

    # ==============================
    # STEP 1: TISSUE MASK (NO CHANGE)
    # ==============================
    tissue = tissue_mask(img)

    out_dir = os.path.join(r"D:\projects\neuro\lowfreq2", name)
    os.makedirs(out_dir, exist_ok=True)

    tissue_only = cv2.bitwise_and(img, img, mask=tissue)
    cv2.imwrite(os.path.join(out_dir, "01_tissue_only.png"), tissue_only)

    # ==============================
    # STEP 2: BLUR MAP ON ORIGINAL IMAGE (FIXED)
    # ==============================
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap = np.abs(lap)

    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        cv2.imwrite(os.path.join(out_dir, f"02_blur_map_{k}x{k}.png"), blur_map)

        # ==============================
        # STEP 3: LOW-FREQ (BLUR) MASK
        # ==============================
        blur_mask = get_blur_mask(blur_map, PERCENTILE_THRESHOLD)

        kernel = np.ones((MORPH_KERNEL_SIZE, MORPH_KERNEL_SIZE), np.uint8)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_CLOSE, kernel)
        blur_mask = cv2.morphologyEx(blur_mask, cv2.MORPH_OPEN, kernel)

        cv2.imwrite(os.path.join(out_dir, f"03_blur_mask_{k}x{k}.png"), blur_mask)

        # ==============================
        # STEP 4: FINAL MASK = TISSUE ∧ BLUR
        # ==============================
        final_mask = cv2.bitwise_and(tissue, blur_mask)

        final_low_freq = cv2.bitwise_and(img, img, mask=final_mask)

        cv2.imwrite(
            os.path.join(out_dir, f"04_low_freq_only_{k}x{k}.png"),
            final_low_freq
        )
#blurmap->tissue mask

Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5


In [20]:
WINDOW_SIZES = [3, 5, 7]
PERCENTILE_THRESHOLD = 20
MORPH_KERNEL_SIZE = 11

for img_path in glob(os.path.join(INPUT_DIR, "*.png")):
    name = os.path.splitext(os.path.basename(img_path))[0]
    print("Processing:", name)

    img = cv2.imread(img_path)
    if img is None:
        continue

    # ==============================
    # STEP 1: TISSUE MASK (NO CHANGE)
    # ==============================
    tissue = tissue_mask(img)

    out_dir = os.path.join(r"D:\projects\neuro\lowfreq3", name)
    os.makedirs(out_dir, exist_ok=True)

    tissue_only = cv2.bitwise_and(img, img, mask=tissue)
    cv2.imwrite(os.path.join(out_dir, "01_tissue_only.png"), tissue_only)

    # ==============================
    # STEP 2: BLUR MAP ON ORIGINAL IMAGE (NO CHANGE)
    # ==============================
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap = np.abs(lap)

    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        cv2.imwrite(os.path.join(out_dir, f"02_blur_map_{k}x{k}.png"), blur_map)

        # ==============================
        # STEP 3: LOW-FREQ (BLUR) MASK (FIXED)
        # ==============================
        blur_mask = get_blur_mask(blur_map, PERCENTILE_THRESHOLD)
        
        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, (11, 11)
        )
        
        blur_mask = cv2.morphologyEx(
            blur_mask, cv2.MORPH_CLOSE, kernel, iterations=2
        )
        blur_mask = cv2.dilate(blur_mask, kernel, iterations=1)
        
        cv2.imwrite(
            os.path.join(out_dir, f"03_blur_mask_{k}x{k}.png"),
            blur_mask
        )


        # ==============================
        # STEP 4: FINAL MASK (CORRECT OR LOGIC)
        # ==============================
        # Remove axons (high-frequency)
        remove_high_freq = cv2.bitwise_not(blur_mask)

        # Remove background
        remove_background = cv2.bitwise_not(tissue)

        # Remove if either condition is true
        remove_mask = cv2.bitwise_or(remove_high_freq, remove_background)

        # Final keep mask = blurred tissue only
        final_mask = cv2.bitwise_not(remove_mask)

        final_low_freq = cv2.bitwise_and(img, img, mask=final_mask)

        cv2.imwrite(
            os.path.join(out_dir, f"04_low_freq_only_{k}x{k}.png"),
            final_low_freq
        )


Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5


In [23]:
WINDOW_SIZES = [3, 5, 7]
AXON_PERCENTILE = 80        # top 15% = axons
MORPH_KERNEL_SIZE = 7

for img_path in glob(os.path.join(INPUT_DIR, "*.png")):
    name = os.path.splitext(os.path.basename(img_path))[0]
    print("Processing:", name)

    img = cv2.imread(img_path)
    if img is None:
        continue

    # ==============================
    # STEP 1: TISSUE MASK
    # ==============================
    tissue = tissue_mask(img)

    out_dir = os.path.join(r"D:\projects\neuro\lowfreq4", name)
    os.makedirs(out_dir, exist_ok=True)

    cv2.imwrite(os.path.join(out_dir, "01_tissue_mask.png"), tissue)

    # ==============================
    # STEP 2: BLUR MAP (GLOBAL)
    # ==============================
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    lap = cv2.Laplacian(gray, cv2.CV_64F)
    lap = np.abs(lap)

    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        cv2.imwrite(os.path.join(out_dir, f"02_blur_map_{k}x{k}.png"), blur_map)

        # ==============================
        # STEP 3: AXON (HIGH-FREQ) MASK
        # ==============================
        axon_thresh = np.percentile(blur_map, AXON_PERCENTILE)

        axon_mask = np.zeros_like(blur_map, dtype=np.uint8)
        axon_mask[blur_map >= axon_thresh] = 255

        kernel = cv2.getStructuringElement(
            cv2.MORPH_ELLIPSE, (MORPH_KERNEL_SIZE, MORPH_KERNEL_SIZE)
        )

        axon_mask = cv2.morphologyEx(
            axon_mask, cv2.MORPH_CLOSE, kernel, iterations=1
        )

        cv2.imwrite(
            os.path.join(out_dir, f"03_axon_mask_{k}x{k}.png"),
            axon_mask
        )

        # ==============================
        # STEP 4: FINAL MASK (REMOVE AXONS + BACKGROUND)
        # ==============================
        remove_background = cv2.bitwise_not(tissue)
        remove_axons = axon_mask

        remove_mask = cv2.bitwise_or(remove_background, remove_axons)
        final_mask = cv2.bitwise_not(remove_mask)

        cv2.imwrite(
            os.path.join(out_dir, f"04_final_mask_{k}x{k}.png"),
            final_mask
        )

        final_output = cv2.bitwise_and(img, img, mask=final_mask)

        cv2.imwrite(
            os.path.join(out_dir, f"05_lowfreq_output_{k}x{k}.png"),
            final_output
        )


Processing: 1
Processing: 2
Processing: 3
Processing: 4
Processing: 5
